In [16]:

"""
synthetic_feature_benchmark.py  (v2, multiclass + miRNA NB simulator)

Generador de datasets sintéticos con soporte (variables relevantes) conocido.
Pensado para evaluar selectores de atributos (feature selection).

Novedades:
- **Clasificación multiclase** (softmax) para modelos de clasificación ('sparse_logistic', 'xor', 'parity', 'madelon_like').
- **Nuevo modelo 'mirna_nb'**: simulación de conteos de miRNA/trascriptómica con
  distribución Binomial Negativa (gamma-poisson), tamaños de biblioteca, genes/miRNAs
  diferencialmente expresados entre clases y módulos de coexpresión opcionales.

Recetarios (model):
- 'sparse_logistic'  -> Clasificación (K clases) con soporte escaso, X gaussiano correlado.
- 'sparse_linear'    -> Regresión lineal esparsa.
- 'friedman1'        -> Regresión no lineal clásica (Friedman #1).
- 'xor' / 'parity'   -> Clasificación (K clases) por interacciones puras.
- 'madelon_like'     -> Clasificación (K clases) estilo MADELON (clústeres en hipercubo).
- 'mirna_nb'         -> Clasificación (K clases) con **datos de conteo** tipo miRNA (NB).

Uso rápido:
    from synthetic_feature_benchmark import make_benchmark_dataset

    X, y, info = make_benchmark_dataset(
        n_samples=1000, n_features=200, n_informative=10,
        model="sparse_logistic", corr="toeplitz", rho=0.5,
        n_classes=3, class_probs=[0.2, 0.5, 0.3], random_state=42
    )

    Xc, yc, info_c = make_benchmark_dataset(
        n_samples=600, n_features=300, n_informative=40,
        model="mirna_nb", n_classes=3, class_probs=[1/3,1/3,1/3],
        effect_size=0.8, # log-fold-change aproximado
        mirna_library_log_mean=11.0, mirna_library_log_sd=0.5,
        mirna_dispersion_scale=1.0, n_modules=5, module_sd=0.3,
        random_state=7
    )

Devuelve:
- X: np.ndarray (n_samples, n_features)  (float en la mayoría de modelos, **int** para 'mirna_nb')
- y: np.ndarray (n_samples,) con etiquetas 0..K-1 (si clasificación) o valores reales (regresión)
- info: dict con metadatos:
        - 'support': máscara booleana de variables estrictamente informativas
        - 'support_with_redundant': máscara booleana incluyendo redundantes
        - 'coef' o 'coef_matrix' (si aplica)
        - 'model', 'n_classes', 'class_probs', 'snr', 'noise_std', 'rho', etc.
        - 'redundant_map': lista de tuplas (j_redundante, indices_fuente)
        - 'repeated_from': lista de tuplas (j_repetida, j_original)
        - 'categorical_idx': índices de columnas categóricas (si se crean)
        - 'is_counts': True para 'mirna_nb'; además incluye 'library_sizes', 'dispersion', 'de_log_fc'
        - 'notes': texto con detalles

Requisitos: NumPy.
"""

from __future__ import annotations
import math
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Any, List, Sequence
import numpy as np

Array = np.ndarray

# ===================== Utilidades generales =====================

def _rng(random_state: Optional[int] = None) -> np.random.Generator:
    if isinstance(random_state, np.random.Generator):
        return random_state
    return np.random.default_rng(random_state)

def _toeplitz_cov(p: int, rho: float) -> Array:
    idx = np.arange(p)
    return rho ** np.abs(idx[:, None] - idx[None, :])

def _block_equicorr_cov(p: int, n_blocks: int, rho_within: float, rho_between: float = 0.0) -> Array:
    if n_blocks <= 0:
        raise ValueError("n_blocks debe ser >= 1")
    sizes = np.full(n_blocks, p // n_blocks, dtype=int)
    sizes[: p % n_blocks] += 1
    idxs = np.cumsum(np.r_[0, sizes])
    Σ = np.full((p, p), rho_between, dtype=float)
    for b in range(n_blocks):
        a, bnd = idxs[b], idxs[b+1]
        Σ[a:bnd, a:bnd] = rho_within
    np.fill_diagonal(Σ, 1.0)
    eigmin = np.linalg.eigvalsh(Σ).min()
    if eigmin <= 1e-6:
        Σ += (1e-6 - eigmin + 1e-8) * np.eye(p)
    return Σ

def _sample_gaussian_features(n: int, p: int, corr: str, rho: float, n_blocks: int, rng: np.random.Generator) -> Array:
    if corr == "independent":
        X = rng.standard_normal((n, p))
    elif corr == "toeplitz":
        Σ = _toeplitz_cov(p, rho)
        L = np.linalg.cholesky(Σ)
        X = rng.standard_normal((n, p)) @ L.T
    elif corr == "block":
        Σ = _block_equicorr_cov(p, n_blocks=n_blocks, rho_within=rho, rho_between=0.0)
        L = np.linalg.cholesky(Σ)
        X = rng.standard_normal((n, p)) @ L.T
    else:
        raise ValueError(f"corr '{corr}' no soportado. Use 'independent', 'toeplitz' o 'block'.")
    return X

def _build_support(p: int, n_informative: int, rng: np.random.Generator, support: Optional[np.ndarray]) -> np.ndarray:
    if support is not None:
        support = np.asarray(support, dtype=bool)
        if support.shape[0] != p:
            raise ValueError("La máscara 'support' debe tener longitud n_features")
        if support.sum() != n_informative:
            n_informative = int(support.sum())
        return support
    idx = rng.choice(p, size=min(n_informative, p), replace=False)
    sup = np.zeros(p, dtype=bool)
    sup[idx] = True
    return sup

def _scale_noise_for_snr(signal: Array, snr: float) -> float:
    var_signal = float(np.var(signal))
    if snr <= 0:
        raise ValueError("snr debe ser > 0")
    sigma = math.sqrt(max(var_signal, 1e-12) / snr)
    return sigma

def _bisect_intercept(eta: Array, target_pi: float, tol: float = 1e-6, max_iter: int = 200) -> float:
    lo, hi = -40.0, 40.0
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        p = 1.0 / (1.0 + np.exp(-(eta + mid)))
        m = p.mean()
        if abs(m - target_pi) < tol:
            return float(mid)
        if m > target_pi:
            hi = mid
        else:
            lo = mid
    return float(mid)

def _softmax(Z: Array) -> Array:
    # Z: (n, K)
    Z = Z - Z.max(axis=1, keepdims=True)
    expZ = np.exp(Z)
    return expZ / expZ.sum(axis=1, keepdims=True)

def _calibrate_softmax_intercepts(eta_no_b: Array, target_probs: Array,
                                  max_iter: int = 500, tol: float = 1e-7, lr: float = 1.0) -> Array:
    """
    Ajusta un vector de interceptos b (longitud K) tal que
    mean(softmax(eta_no_b + b)) ≈ target_probs.
    """
    n, K = eta_no_b.shape
    b = np.zeros(K, dtype=float)
    target = np.asarray(target_probs, dtype=float)
    target = target / target.sum()
    for _ in range(max_iter):
        P = _softmax(eta_no_b + b)
        grad = P.mean(axis=0) - target     # dL/db ~ diferencia de promedios
        gnorm = np.linalg.norm(grad, ord=2)
        if gnorm < tol:
            break
        # paso con pequeño damping
        b -= lr * grad
        # retirar la media para evitar indeterminación (sum shift invariance)
        b -= b.mean()
    return b

def _add_redundant_features(X: Array, support_idx: np.ndarray, n_redundant: int,
                            noise_std: float, rng: np.random.Generator):
    n, p = X.shape
    red_map = []
    if n_redundant <= 0:
        return X, red_map
    base_idx = np.array(support_idx, copy=False)
    for _ in range(n_redundant):
        k = int(rng.integers(low=1, high=min(4, len(base_idx)) + 1))
        choose = rng.choice(base_idx, size=k, replace=False)
        weights = rng.normal(0, 1, size=k)
        new_col = X[:, choose] @ weights + rng.normal(0, noise_std * 0.1, size=n)
        X = np.column_stack([X, new_col])
        red_map.append((X.shape[1]-1, list(map(int, choose))))
    return X, red_map

def _add_repeated_features(X: Array, n_repeats: int, rng: np.random.Generator):
    n, p = X.shape
    rep = []
    if n_repeats <= 0:
        return X, rep
    src = rng.integers(low=0, high=p, size=n_repeats)
    for s in src:
        X = np.column_stack([X, X[:, s]])
        rep.append((X.shape[1]-1, int(s)))
    return X, rep

def _add_categorical_noise(X: Array, n_categorical: int, n_categories: int, rng: np.random.Generator):
    n, _ = X.shape
    cat_idx = []
    for _ in range(n_categorical):
        col = rng.integers(0, n_categories, size=n).astype(float)
        X = np.column_stack([X, col])
        cat_idx.append(X.shape[1]-1)
    return X, cat_idx

# ===================== Utilidades NB (miRNA/scRNA) =====================

def _sample_nb_means(n_genes: int, rng: np.random.Generator,
                     mean_log_expr: float = 1.5, sd_log_expr: float = 1.0) -> Array:
    # Intensidades base por gen (proporciones) ~ LogNormal
    base_int = rng.lognormal(mean=mean_log_expr, sigma=sd_log_expr, size=n_genes)
    base_int = np.maximum(base_int, 1e-12)
    pi_g = base_int / base_int.sum()  # proporciones por gen
    return pi_g

def _sample_library_sizes(n_samples: int, rng: np.random.Generator,
                          log_mean: float = 11.0, log_sd: float = 0.5) -> Array:
    # Tamaños de biblioteca (total counts por célula/muestra) ~ LogNormal
    L = rng.lognormal(mean=log_mean, sigma=log_sd, size=n_samples)
    return L

def _sample_nb_counts(mu: Array, disp: Array, rng: np.random.Generator) -> Array:
    """
    Muestras de NB con parametrización (mean=mu, dispersion=alpha),
    Var = mu + alpha * mu^2. Se implementa como Gamma-Poisson:
      lambda ~ Gamma(shape=1/alpha, scale=mu*alpha),  y ~ Poisson(lambda).
    - mu: (n, p)
    - disp: (p,)  (alpha >= 0)
    """
    n, p = mu.shape
    alpha = np.asarray(disp, dtype=float)
    alpha = np.maximum(alpha, 1e-12)
    r = 1.0 / alpha
    # Broadcasting shape/scale por columna
    lam = rng.gamma(shape=r, scale=mu * alpha, size=(n, p))
    y = rng.poisson(lam)
    return y

# ===================== Interfaz principal =====================

def make_benchmark_dataset(
    n_samples: int = 1000,
    n_features: int = 100,
    n_informative: int = 10,
    # Tarea y modelo
    task: str = "classification",            # Se infiere a partir de 'model'; aquí es informativo
    model: str = "sparse_logistic",          # 'sparse_logistic', 'sparse_linear', 'friedman1', 'xor', 'parity', 'madelon_like', 'mirna_nb'
    # Estructura de X (gaussiano)
    corr: str = "independent",               # 'independent', 'toeplitz', 'block'
    rho: float = 0.3,                        # intensidad de correlación (Toeplitz/Block)
    n_blocks: int = 5,                       # nº de bloques si corr='block'
    # Señal/ruido y efectos
    snr: float = 5.0,                        # SNR para regresión; cuantos "signal variances" por "noise variance"
    effect_size: float = 1.0,                # magnitud de coeficientes (lineal) o log-fold-change (miRNA)
    heteroscedastic: float = 0.0,            # 0 => homoscedástico; >0 => var(noise) crece con |Xβ|
    # Clasificación
    n_classes: int = 2,
    class_balance: float = 0.5,              # mantenido por compatibilidad (binario); usar 'class_probs' preferentemente
    class_probs: Optional[Sequence[float]] = None,  # distribución objetivo de clases
    # Redundantes/ruido extra
    n_redundant: int = 0,
    n_repeated: int = 0,
    n_categorical: int = 0,
    n_categories: int = 3,
    # Opcional: fijar soporte
    support_mask: Optional[np.ndarray] = None,
    # Parámetros específicos miRNA
    mirna_library_log_mean: float = 11.0,
    mirna_library_log_sd: float = 0.5,
    mirna_mean_log_expr: float = 1.5,
    mirna_sd_log_expr: float = 1.0,
    mirna_dispersion_scale: float = 1.0,     # escala para la dispersión (alpha)
    dropout_rate: float = 0.0,               # prob. de "dropout" (0 típico para miRNA; >0 para simular scRNA)
    n_modules: int = 0,                      # nº de módulos de coexpresión (0 => sin módulos)
    module_sd: float = 0.3,                  # desviación (log-escala) de factores por muestra para cada módulo
    random_state: Optional[int] = None
) -> Tuple[Array, Array, Dict[str, Any]]:
    """
    Crea un dataset sintético con soporte conocido de variables relevantes.
    Permite clasificación multiclase en modelos de clasificación, y un modelo
    específico de conteos NB para miRNA ('mirna_nb').

    Retorna (X, y, info).
    """
    rng = _rng(random_state)

    # ====== Preparación de probabilidades de clase ======
    if class_probs is None:
        if n_classes == 2:
            class_probs = [1 - class_balance, class_balance]
        else:
            class_probs = [1.0 / n_classes] * n_classes
    class_probs = np.asarray(class_probs, dtype=float)
    if len(class_probs) != n_classes:
        raise ValueError("class_probs debe tener longitud n_classes")
    class_probs = class_probs / class_probs.sum()

    # ====== MODELOS ESPECÍFICOS ======
    if model in ("sparse_logistic", "sparse_linear"):
        # 1) Generamos X
        X = _sample_gaussian_features(n_samples, n_features, corr=corr, rho=rho, n_blocks=n_blocks, rng=rng)

        # 2) Soporte
        support = _build_support(n_features, n_informative, rng, support_mask)

        if model == "sparse_linear":
            # Coeficientes sparse para regresión
            coef = np.zeros(n_features)
            magnitudes = rng.uniform(0.5, 1.5, size=support.sum()) * effect_size
            signs = rng.choice([-1.0, 1.0], size=support.sum())
            coef[support] = magnitudes * signs
            eta = X @ coef
            sigma = _scale_noise_for_snr(eta, snr=snr)
            if heteroscedastic > 0:
                noise = rng.normal(0, sigma, size=n_samples) * (1.0 + heteroscedastic * np.abs(eta))
            else:
                noise = rng.normal(0, sigma, size=n_samples)
            y = eta + noise
            base_noise_std = float(np.std(eta))

            info = {
                "model": model, "task": "regression",
                "support": support, "support_with_redundant": support.copy(),
                "coef": coef, "coef_matrix": None,
                "snr": snr, "noise_std": base_noise_std, "corr": corr, "rho": rho, "n_blocks": n_blocks,
                "n_classes": None, "class_probs": None,
                "notes": "Diseño gaussiano con soporte escaso; coeficientes aleatorios (regresión)."
            }

        else:
            # Clasificación multiclase (softmax) con coeficientes compartiendo el mismo soporte
            K = int(n_classes)
            # Matriz de coeficientes (p x K); ceros fuera del soporte
            coef_matrix = np.zeros((n_features, K))
            magnitudes = rng.uniform(0.5, 1.5, size=(support.sum(), K)) * effect_size
            signs = rng.choice([-1.0, 1.0], size=(support.sum(), K))
            coef_matrix[support, :] = magnitudes * signs
            eta_no_b = X @ coef_matrix  # (n, K)
            # Calibrar interceptos para alcanzar class_probs deseadas
            b = _calibrate_softmax_intercepts(eta_no_b, class_probs)
            eta = eta_no_b + b  # broadcasting
            P = _softmax(eta)
            # Muestreamos clases
            cumP = np.cumsum(P, axis=1)
            r = rng.random(n_samples)[:, None]
            y = (cumP < r).sum(axis=1)
            info = {
                "model": model, "task": "classification",
                "support": support, "support_with_redundant": support.copy(),
                "coef": None, "coef_matrix": coef_matrix,
                "snr": None, "noise_std": float(np.std(eta_no_b)),
                "corr": corr, "rho": rho, "n_blocks": n_blocks,
                "n_classes": K, "class_probs": class_probs,
                "notes": "Clasificación softmax con soporte escaso y X gaussiano."
            }

    elif model == "friedman1":
        p0 = max(n_features, 10)
        X = rng.uniform(0.0, 1.0, size=(n_samples, p0))
        y_signal = 10*np.sin(np.pi * X[:,0] * X[:,1]) + 20*(X[:,2] - 0.5)**2 + 10*X[:,3] + 5*X[:,4]
        sigma = _scale_noise_for_snr(y_signal, snr=snr)
        y = y_signal + rng.normal(0, sigma, size=n_samples)
        support = np.zeros(p0, dtype=bool); support[:5] = True
        info = {
            "model": "friedman1", "task": "regression",
            "support": support[:n_features], "support_with_redundant": support[:n_features].copy(),
            "coef": None, "coef_matrix": None, "snr": snr, "noise_std": sigma,
            "corr": "independent", "rho": 0.0, "n_blocks": None,
            "n_classes": None, "class_probs": None,
            "notes": "Friedman #1: no lineal aditivo con 5 variables relevantes (x1..x5)."
        }
        if p0 != n_features:
            if n_features < p0:
                X = X[:, :n_features]
            else:
                extra = rng.uniform(0.0, 1.0, size=(n_samples, n_features - p0))
                X = np.concatenate([X, extra], axis=1)

    elif model in ("xor", "parity"):
        k = int(max(2, n_informative))
        bits = rng.integers(0, 2, size=(n_samples, k))
        jitter = rng.normal(0, 0.1, size=(n_samples, k))
        informative = bits.astype(float) + jitter
        p_noise = max(0, n_features - k)
        noise = rng.standard_normal((n_samples, p_noise)) if p_noise > 0 else None
        X = np.concatenate([informative, noise], axis=1) if noise is not None else informative
        # Etiquetas multiclase: suma de bits modulo K
        K = int(n_classes)
        y = (bits.sum(axis=1) % K).astype(int)
        support = np.zeros(X.shape[1], dtype=bool); support[:k] = True
        info = {
            "model": model, "task": "classification",
            "support": support, "support_with_redundant": support.copy(),
            "coef": None, "coef_matrix": None, "snr": None, "noise_std": 0.1,
            "corr": "independent", "rho": 0.0, "n_blocks": None,
            "n_classes": K, "class_probs": class_probs,
            "notes": "Etiqueta = (paridad/XOR de k bits) mod K, con jitter."
        }

    elif model == "madelon_like":
        base_dim = 5
        vertices = np.array(np.meshgrid(*[[-1, 1]]*base_dim)).T.reshape(-1, base_dim)
        # Asignación de clases a los 32 vértices, intentando balancear K clases
        K = int(n_classes)
        labels = np.repeat(np.arange(K), repeats=len(vertices)//K)
        if labels.size < len(vertices):
            labels = np.r_[labels, np.arange(len(vertices) - labels.size) % K]
        labels = labels[:len(vertices)]
        labels = rng.permutation(labels)  # aleatorizar asignación vértice->clase
        # Generación por clúster
        centers_idx = rng.integers(0, len(vertices), size=n_samples)
        centers = vertices[centers_idx]
        y = labels[centers_idx]
        sep = 2.0
        base = centers * sep + rng.normal(0, 1.0, size=(n_samples, base_dim))
        # Combinaciones lineales (redundantes) de las 5 bases
        n_lin = min(15, max(0, n_features - base_dim))
        lin_cols = []
        red_map = []
        for j in range(n_lin):
            w = rng.normal(0, 1, size=base_dim)
            col = base @ w + rng.normal(0, 0.2, size=n_samples)
            lin_cols.append(col)
        if lin_cols:
            lin_cols = np.column_stack(lin_cols)
            X = np.column_stack([base, lin_cols])
            red_map = [(base_dim + j, list(range(base_dim))) for j in range(n_lin)]
        else:
            X = base
        if X.shape[1] < n_features:
            X = np.column_stack([X, rng.standard_normal((n_samples, n_features - X.shape[1]))])
        support = np.zeros(X.shape[1], dtype=bool); support[:base_dim] = True
        support_with_redundant = support.copy()
        support_with_redundant[base_dim:base_dim + n_lin] = True
        info = {
            "model": "madelon_like", "task": "classification",
            "support": support, "support_with_redundant": support_with_redundant,
            "coef": None, "coef_matrix": None, "snr": None, "noise_std": 1.0,
            "corr": "independent", "rho": 0.0, "n_blocks": None,
            "redundant_map": red_map,
            "n_classes": K, "class_probs": class_probs,
            "notes": f"Clústeres en hipercubo 5D; 5 informativas + {n_lin} combinaciones lineales + ruido, con {K} clases."
        }

    elif model == "mirna_nb":
        # ===== Simulación de conteos NB (miRNA) =====
        K = int(n_classes)
        # 1) Asignar clases a muestras según class_probs
        cum = np.cumsum(class_probs)
        r = rng.random(n_samples)
        y = np.searchsorted(cum, r).astype(int)

        p = n_features
        # 2) Proporciones base por gen/miRNA
        pi_g = _sample_nb_means(p, rng, mean_log_expr=mirna_mean_log_expr, sd_log_expr=mirna_sd_log_expr)  # (p,)
        # 3) Tamaños de biblioteca por muestra
        L = _sample_library_sizes(n_samples, rng, log_mean=mirna_library_log_mean, log_sd=mirna_library_log_sd)  # (n,)
        # 4) Módulos de coexpresión opcionales (en log-escala multiplicativa)
        module_of = None
        sample_module_effect = None
        if n_modules and n_modules > 0:
            module_of = rng.integers(0, n_modules, size=p)
            sample_module_effect = rng.normal(0.0, module_sd, size=(n_samples, n_modules))
        # 5) Soporte DE: genes informativos (DE en al menos una clase)
        support = _build_support(p, n_informative, rng, support_mask)
        # 6) Efectos por clase (log-fold-changes); clase 0 referencia (0)
        de_log_fc = np.zeros((p, K), dtype=float)
        # Para genes informativos, aplicar efectos aleatorios por clase (centrados)
        for g in np.flatnonzero(support):
            # efectos por clase alrededor de 0, con escala effect_size
            de_log_fc[g, 1:] = rng.normal(0.0, effect_size, size=K-1)
            # opcional: hacer algunos efectos con signo consistente por clase
        # 7) Medias mu_{i,g} = L_i * pi_g * exp(module_effect + class_effect)
        log_mu = (np.log(L)[:, None] + np.log(pi_g)[None, :])  # (n,p)
        if n_modules and n_modules > 0:
            # sumar efecto del módulo correspondiente a cada gen
            m_idx = module_of[None, :]                 # (1,p)
            mod_eff = np.take_along_axis(sample_module_effect, m_idx, axis=1)  # (n,p)
            log_mu = log_mu + mod_eff
        # efecto por clase sobre genes informativos
        class_eff = de_log_fc[y, :]                    # (n,K) but we need per gene -> expand
        # Convertir a matriz (n,p): por muestra, añadir efecto de su clase para cada gen
        # de_log_fc es (p,K); tomamos columna y_i
        class_shift_per_gene = de_log_fc[:, y].T      # (n,p)
        log_mu = log_mu + class_shift_per_gene
        mu = np.exp(log_mu)                            # medias NB
        # 8) Dispersión por gen: alpha >= 0 (Var = mu + alpha*mu^2)
        #    Usamos alpha_g ~ LogNormal(-1.5, 0.5) escalada
        alpha_g = np.exp(rng.normal(-1.5, 0.5, size=p)) * float(mirna_dispersion_scale)
        # 9) Muestras de conteo
        X = _sample_nb_counts(mu, alpha_g, rng)
        # 10) Dropout opcional
        if dropout_rate and dropout_rate > 0.0:
            mask = rng.random(size=X.shape) < float(dropout_rate)
            X[mask] = 0
        info = {
            "model": "mirna_nb", "task": "classification",
            "support": support, "support_with_redundant": support.copy(),
            "coef": None, "coef_matrix": None,
            "snr": None, "noise_std": None, "corr": None, "rho": None, "n_blocks": None,
            "n_classes": K, "class_probs": class_probs,
            "is_counts": True,
            "library_sizes": L, "base_proportions": pi_g,
            "dispersion": alpha_g, "de_log_fc": de_log_fc,
            "module_of": module_of, "module_sd": module_sd,
            "notes": "Conteos NB por miRNA con tamaños de biblioteca, DE multiclase y módulos de coexpresión opcionales."
        }

    else:
        raise ValueError(f"model '{model}' no soportado.")

    # ====== OPCIONALES (no aplican a 'mirna_nb' ni 'friedman1' salvo donde tenga sentido) ======
    redundant_map = list(info.get("redundant_map", []))
    repeated_from = []

    if model not in ("madelon_like", "friedman1", "mirna_nb"):
        support_idx = np.flatnonzero(info["support"])
        if n_redundant > 0:
            X, red_map_extra = _add_redundant_features(X, support_idx, n_redundant, noise_std=0.1, rng=rng)
            redundant_map.extend(red_map_extra)
            supp = info["support_with_redundant"]
            if supp.shape[0] < X.shape[1]:
                supp = np.r_[supp, np.zeros(X.shape[1] - supp.shape[0], dtype=bool)]
            for j,_ in red_map_extra:
                supp[j] = True
            info["support_with_redundant"] = supp

    if model not in ("mirna_nb",):
        if n_repeated > 0:
            X, rep = _add_repeated_features(X, n_repeated, rng)
            repeated_from.extend(rep)

        categorical_idx: List[int] = []
        if n_categorical > 0:
            X, categorical_idx = _add_categorical_noise(X, n_categorical, n_categories, rng)
        else:
            categorical_idx = info.get("categorical_idx", [])
    else:
        categorical_idx = info.get("categorical_idx", [])

    if X.shape[1] < n_features and model not in ("mirna_nb",):
        X = np.column_stack([X, rng.standard_normal((n_samples, n_features - X.shape[1]))])

    # Actualizamos info y salidas
    p_final = X.shape[1]
    for key in ("support", "support_with_redundant"):
        mask = info.get(key, None)
        if mask is None:
            continue
        if mask.shape[0] < p_final:
            pad = np.zeros(p_final - mask.shape[0], dtype=bool)
            mask = np.r_[mask, pad]
        else:
            mask = mask[:p_final]
        info[key] = mask

    info["redundant_map"] = redundant_map
    info["repeated_from"] = repeated_from
    info["categorical_idx"] = categorical_idx
    info["n_features_final"] = p_final

    # Tipo de retorno (mantener conteos como int si es miRNA)
    if info.get("is_counts", False):
        return X.astype(int), y, info
    else:
        return X.astype(float), y, info


In [ ]:


X, y, info = make_benchmark_dataset(
    n_samples=900,
    n_features=200,
    n_informative=10,
    n_classes=4, 
    class_probs=[0.1, 0.2, 0.3, 0.4],
    task="classification",                # o "regression"
    model="sparse_logistic",              # 'sparse_logistic', 'sparse_linear', 'friedman1', 'xor', 'parity', 'madelon_like'
    corr="toeplitz",                      # 'independent', 'toeplitz', 'block'
    rho=0.5,                              # intensidad de correlación
    n_blocks=5,                           # si corr='block'
    snr=5.0,                              # para regresión
    effect_size=1.0,
    heteroscedastic=0.0,                  # >0 para ruido heteroscedástico en regresión
    class_balance=0.5,                    # prevalencia deseada en clasificación
    n_redundant=5,                        # combinaciones lineales extra de las informativas
    n_repeated=2,                         # duplicados exactos de columnas
    n_categorical=3, n_categories=4,      # variables categóricas de ruido (0..K-1)
    random_state=42
)


In [17]:
import pandas as pd

X, y, info = make_benchmark_dataset(
    n_samples=2000, n_features=200,  # 5 base + 15 redundantes + 480 ruido (aprox.)
    task="classification", model="madelon_like",
    random_state=3,
    n_classes=3, 
    class_probs=[1/3, 1/3, 1/3]
)


df = pd.DataFrame(X)
df['target'] = y
df.to_csv('madelon_like_dataset.csv', index=False)

In [19]:
info

{'model': 'madelon_like',
 'task': 'classification',
 'support': array([ True,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False

In [12]:
Xc, yc, info_c = make_benchmark_dataset(
    n_samples=600, n_features=300, n_informative=20,
    model="mirna_nb",
    n_classes=3, 
    class_probs=[1/3, 1/3, 1/3],
    # Intensidades y biblioteca
    mirna_mean_log_expr=1.5, mirna_sd_log_expr=1.0,
    mirna_library_log_mean=11.0, mirna_library_log_sd=0.5,
    # Dispersión y efecto diferencial
    mirna_dispersion_scale=1.0,
    effect_size=0.8,   # ~desv. típica de log-FC por clase en genes informativos
    # Coexpresión opcional
    n_modules=5, module_sd=0.3,
    # Dropout (normalmente 0 en miRNA; >0 para imitar scRNA)
    dropout_rate=0.0,
    random_state=7
)

df_c = pd.DataFrame(Xc)
df_c['target'] = yc
df_c.to_csv('mirna_nb_dataset.csv', index=False)


In [13]:
import numpy as np

true_positions = np.where(info_c["support"])[0]
true_positions

array([  3,  10,  36,  38,  50,  57,  58,  71, 115, 141, 172, 188, 215,
       236, 237, 249, 273, 279, 280, 288])

In [15]:
import numpy as np

true_positions = np.where(info_c["support_with_redundant"])[0]
true_positions

array([  3,  10,  36,  38,  50,  57,  58,  71, 115, 141, 172, 188, 215,
       236, 237, 249, 273, 279, 280, 288])